In [1]:
from sklearn import neighbors
from sklearn import preprocessing
from arch.bootstrap import StationaryBootstrap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import math
import sys
import random
from itertools import chain

In [3]:
#read training and cross validation set
total = pd.read_csv("time_space.csv")
n = total.shape[0]
num_tract = 1758

total = total.drop(total.columns[0], axis = 1) #delete index

tract_id = total['GEOID']
total = total.drop(total.columns[0], axis = 1) #delete tract_geoid
    
mp_date = total['MP_DATE']
origin = dt.datetime.strptime(total.loc[0, 'MP_DATE'],'%Y-%m-%d').date()
for i in range(n):
    now = dt.datetime.strptime(total.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    total.loc[i, 'MP_DATE'] = (now - origin).days/30.0    
values = total['values']
total = total.drop(total.columns[3], axis = 1) #delete values

test = pd.read_csv("test_data.csv")
m = test.shape[0]
test = test.drop(test.columns[0], axis = 1) #delete index
test_geoid = test['GEOID']
test = test.drop(test.columns[0], axis = 1) #delete GEOID
for i in range(m):
    now = dt.datetime.strptime(test.loc[i, 'MP_DATE'],'%Y-%m-%d').date()
    test.loc[i, 'MP_DATE'] = (now - origin).days/30.0

In [4]:
#scale dates
#total['MP_DATE'] = preprocessing.scale(total['MP_DATE'])
#test['MP_DATE'] = preprocessing.scale(test['MP_DATE'])
s = 20

In [3]:
for blocksize in range(25):
    print("Current Block Size: "+str(blocksize))
    
    #get 10 block bootstrapping samples
    for j in range(num_tract):
        sub_data = total.iloc[63*j:63*(j+1)]
        bs = StationaryBootstrap(blocksize, sub_data)
        if j == 0:
            train_lst = [data[0][0] for data in bs.bootstrap(10)]
        else:
            i = 0
            for data in bs.bootstrap(10):
                train_lst[i] = train_lst[i].append(data[0][0])
                i += 1
    print("Finish Bootstrapping")

            
    #get unselected samples as my cross validation
    ind_all = set(total.index)
    ind_diff = []
    possible_oob = set(chain.from_iterable([list(range(63*k+24,63*(k+1)+1)) for k in range(1758)]))
    d_oob_learner = dict()
    d_oob_cv = dict()

    for i in range(s):
        ind_unselected = list(possible_oob.intersection(ind_all - set(train_lst[i].index)))
        print(len(ind_unselected))
        for item in ind_unselected:
            try:
                d_oob_learner[item] += [i]
            except:
                d_oob_learner[item] = [i]
        d_oob_cv[i] = ind_unselected
    print("Finish collecting feasible OOB for CV and prediction")
    
    
    #find best k for each bootstrapping sample --> model parameter
    best_k_lst = []
    best_mae_lst = []
    symbol_lst = []
    y_ = []
    for i in range(s):
        train_y = train_lst[i]['dv'] #get dependent variable
        train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables

        best_mae = sys.maxsize
        for k in range(1, 32, 5):
            knn = neighbors.KNeighborsRegressor(k, weights ='distance')
            y_ = []
            y_mean = 0
            y_symbol = 0
            cv_times = range(51,62)
            for cv_time in cv_times:
                train_id = list(chain.from_iterable([list(range(63*k,(63*k+cv_time))) for k in range(num_tract)]))
                test_id = [63*k+cv_time for k in range(num_tract)]
            #print(test_id)
                y_predict = knn.fit(train_x.iloc[train_id], train_y.iloc[train_id]).predict(train_x.iloc[test_id])
                y_ += [y_predict]
                mae = np.mean(abs(y_predict - train_y.iloc[test_id]))
                y_mean += mae
                y_symbol += sum(y_predict*train_y.iloc[test_id]>=0)/len(y_predict)
            y_mean = y_mean / len(cv_times)
            y_symbol = y_symbol / len(cv_times)
            if y_mean < best_mae:
                best_mae = y_mean
                best_k = k
                symbol = y_symbol
                print((i, k, best_mae))
        best_mae_lst.append(best_mae)
        best_k_lst.append(best_k)
        symbol_lst.append(symbol)



In [5]:
blocksize = 12
#get 10 block bootstrapping samples
for j in range(num_tract):
    sub_data = total.iloc[63*j:63*(j+1)]
    bs = StationaryBootstrap(blocksize, sub_data)
    if j == 0:
        train_lst = [data[0][0] for data in bs.bootstrap(10)]
    else:
        i = 0
        for data in bs.bootstrap(10):
            train_lst[i] = train_lst[i].append(data[0][0])
            i += 1
print("Finish Bootstrapping")

            
#get unselected samples as my cross validation
#ind_all = set(total.index)
#ind_diff = []
#possible_oob = set(chain.from_iterable([list(range(63*k+24,63*(k+1)+1)) for k in range(1758)]))
#d_oob_learner = dict()
#d_oob_cv = dict()

#for i in range(10):
#    ind_unselected = list(possible_oob.intersection(ind_all - set(train_lst[i].index)))
#    print(len(ind_unselected))
#    for item in ind_unselected:
#        try:
#            d_oob_learner[item] += [i]
#        except:
#            d_oob_learner[item] = [i]
#    d_oob_cv[i] = ind_unselected
#print("Finish collecting feasible OOB for CV and prediction")


Finish Bootstrapping


In [19]:
#find best k for each bootstrapping sample --> model parameter
best_k_lst = []
best_mae_lst = []
symbol_lst = []
y_ = []
loss_per_tract = []
for i in range(s):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables

    best_mae = sys.maxsize
    for k in range(1, 11, 1):
        knn = neighbors.KNeighborsRegressor(k, weights ='distance')
        y_ = []
        y_mean = 0
        y_symbol = 0
        cv_times = range(51,62)
        mae_tract_k = []
        
        for cv_time in cv_times:
            train_id = list(chain.from_iterable([list(range(63*k,(63*k+cv_time))) for k in range(num_tract)]))
            test_id = [63*k+cv_time for k in range(num_tract)]
            #print(test_id)
            y_predict = knn.fit(train_x.iloc[train_id], train_y.iloc[train_id]).predict(train_x.iloc[test_id])
            y_ += [y_predict]
            mae_tract_k += [abs(y_predict - train_y.iloc[test_id])]
            mae = np.mean(abs(y_predict - train_y.iloc[test_id]))
            y_mean += mae
            y_symbol += sum(y_predict*train_y.iloc[test_id]>=0)/len(y_predict)
            
        mae_tract_k = np.array(mae_tract_k)
        mae_tract_k = list(np.mean(mae_tract_k,axis = 0))
        y_mean = y_mean / len(cv_times)
        y_symbol = y_symbol / len(cv_times)
        if y_mean < best_mae:
            best_mae = y_mean
            best_k = k
            symbol = y_symbol
            mae_tract = mae_tract_k
            print((i, k, best_mae))
    #print(mae_tract)
    #mae_tract = np.array(mae_tract)
    #print(mae_tract)
    #mae_tract = list(np.mean(mae_tract,axis = 0))

    best_mae_lst.append(best_mae)
    best_k_lst.append(best_k)
    symbol_lst.append(symbol)
    print(mae_tract)
    loss_per_tract.append([mae_tract])
    
loss_per_tract = np.array(loss_per_tract)
loss_per_tract = list(np.mean(loss_per_tract,axis=0))

(0, 1, 1.2738354734948976)
(0, 2, 1.2284398882276528)
(0, 3, 1.2232430596783017)
[2.6688907957739483, 0.0039921281905079212, 2.236077484790206e-13, 0.0, 0.0, 0.0, 0.0, 2.788415695506965, 0.0, 2.91447925609405, 1.0092936587501423e-17, 0.91858718952110863, 2.8638326963760807, 5.0060208503762993e-13, 9.9269330129784938e-13, 1.5139404881252134e-17, 2.271541540724539e-13, 4.3552420716095233, 0.70922486148835451, 1.2616170734376779e-18, 5.0464682937507117e-18, 0.0, 0.24290275384949911, 0.71291527316169145, 0.88038956650870326, 0.042434142360931931, 67.331185996129591, 1.0092936587501423e-17, 5.0464682937507117e-18, 0.054915581099369032, 7.2800351605647767e-14, 0.044436402372236318, 1.4446504598646987e-11, 0.0, 1.5237715792715514, 0.37282848926518303, 0.0, 0.66709407760163331, 0.12197334475035994, 0.50968153957163542, 2.3509356133607779, 6.7398402747145978, 2.6917861878866295e-13, 1.4193192076173875e-16, 0.5034977191485388, 1.8924256101565167e-17, 0.0, 1.0092936587501423e-17, 0.20399408890908

In [13]:
best_k_lst

[3, 1, 1, 2, 1, 1, 3, 6, 8, 5]

TypeError: 'numpy.float64' object is not iterable

In [28]:
#draw correct rate
plt.figure(figsize=(8,6), dpi=60)
plt.plot(range(10), symbol_lst, 'k')
plt.savefig("final_results/accuracy.png", dpi = 60)
np.mean(symbol_lst)

C:\Users\green\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


0.96023373668424861

In [29]:
#draw MAE
plt.figure(figsize=(8,6), dpi=60)
plt.plot(range(10), best_mae_lst, 'k')
plt.savefig("final_results/MAE.png", dpi = 60)
np.mean(best_mae_lst)

C:\Users\green\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


1.9989202309305589

In [14]:
#get predictions
y_pre_lst = []
for 
for i in range(10):
    train_y = train_lst[i]['dv'] #get dependent variable
    train_x = train_lst[i].drop(train_lst[i].columns[3], axis = 1) #delete dependent variables
    knn = neighbors.KNeighborsRegressor(best_k_lst[i], weights ='distance')
    y_pre = knn.fit(train_x, train_y).predict(test)
    y_pre_lst.append(y_pre)
y_pre_mat = np.array(y_pre_lst)
y_final_pre = np.mean(y_pre_mat, 0)

In [15]:
# draw graphs to show predictions of each tract
for i in range(1758):
    filename = "figure/%d.png" % i
    plt.figure(figsize=(8,6), dpi=60)
    plt.plot(range(12), y_final_pre[i*12:(i+1)*12], 'k')
    plt.savefig(filename, dpi = 60)

In [25]:
#output average increase rates for each tract
unique_tract = list((set(tract_id)))
tract_rate = []
for i in range(1758):
    future_rate = y_final_pre[i*12:(i+1)*12]
    tract_rate.append(np.mean(future_rate))
data = {'Future Annual Increase Rate': tract_rate,
        'TRACT_GEOID': unique_tract}
df = pd.DataFrame(data, index = range(1, 1759, 1))
df.to_csv('final_prediction.csv')

C:\Users\green\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [17]:
# final recommendations
recommend = dict()
recommend['invest'] = []
recommend['hold'] = []
recommend['divest'] = []

    if future_rate[1] < future_rate[0]:
        recommend['divest'].append(unique_tract[i])
    elif future_rate[0] > 0.2:
        recommend['invest'].append(unique_tract[i])
    else:
        recommend['hold'].append(unique_tract[i])

In [ ]:
#output invest decisions for each tract
n_1 = len(recommend['divest'])
n_2 = len(recommend['hold'])
n_3 = len(recommend['invest'])
n_max = max(n_1, n_2, n_3)

In [31]:
y_final_mat

NameError: name 'y_final_mat' is not defined